sample_data


In [3]:
!virtualenv myenv

/bin/bash: line 1: virtualenv: command not found


In [4]:
!source myenv/bin/activate

/bin/bash: line 1: myenv/bin/activate: No such file or directory


In [ ]:
%%capture
!pip install -q vllm transformers accelerate langchain langchain-openai pydantic

!pip install langchain_google_genai langchain langchain-community langchain_google_genai
!pip install pypdf
!pip install langchain-text-splitters langchain_huggingface

In [2]:

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

from pydantic import BaseModel
from typing import List

from langchain_core.output_parsers import PydanticOutputParser
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM,pipeline
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace

In [3]:
# !huggingface-cli login


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [6]:

# model_name = "Qwen/Qwen2.5-3B-Instruct"
# model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
model_name = "mistralai/Mistral-7B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [7]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,  # Increased for complete responses
    temperature=0.0,
    do_sample=False,
    return_full_text=False,
    eos_token_id=tokenizer.eos_token_id,
)

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [8]:
llm = HuggingFacePipeline(pipeline=pipe)


In [9]:
pdf_path = "/content/fine_tune_data.pdf"

In [10]:

# Ensure the PDF file exists for the loader to work
if not os.path.exists(pdf_path):
    print(f"Error: The file '{pdf_path}' was not found. Please upload your PDF file or provide a correct path.")
else:
    try:
        loader = PyPDFLoader(pdf_path)
        documents = loader.load()
        print(f"Successfully loaded {len(documents)} pages from {pdf_path}")


    except Exception as e:
        print(f"An error occurred while loading the PDF: {e}")


Successfully loaded 17 pages from /content/fine_tune_data.pdf


In [11]:
# =============== 2. Split into chunks =========
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=60
)

docs = text_splitter.split_documents(documents)


In [12]:
class QAPair(BaseModel):
    question: str
    answer: str


class QAPairs(BaseModel):
    qna_pairs: List[QAPair]


In [13]:
parser = PydanticOutputParser(pydantic_object=QAPairs)


In [14]:
format_instructions = parser.get_format_instructions()

In [15]:
# prompt = PromptTemplate(
#     template="""You generate training data.

# Generate EXACTLY 5 question–answer pairs from the text.

# Output format rules:
# - Output ONLY valid JSON
# - JSON must match the schema
# - No markdown
# - No explanations
# - No trailing text

# TEXT:
# {chunk}

# {format_instructions}
# """,
#     input_variables=["chunk"],
#     partial_variables={"format_instructions": format_instructions},
# )


In [16]:
prompt = PromptTemplate(
    template="""You are a data extraction system.

Generate EXACTLY 5 question–answer pairs.

CRITICAL RULES:
- Answers MUST copy wording directly from the text
- Do NOT paraphrase
- Do NOT summarize
- Use full sentences from the text
- Output ONLY valid JSON
- No markdown
- No explanations

TEXT:
{chunk}

JSON FORMAT:
{format_instructions}
""",
    input_variables=["chunk"],
    partial_variables={"format_instructions": format_instructions},
)


In [17]:
final_prompt = prompt.invoke({"chunk": docs[0].page_content})

In [18]:
# final_prompt = prompt.invoke({"chunk": docs[0].page_content,
#                               "format_instructions": format_instructions})

In [19]:
print(final_prompt)

text='You are a data extraction system.\n\nGenerate EXACTLY 5 question–answer pairs.\n\nCRITICAL RULES:\n- Answers MUST copy wording directly from the text\n- Do NOT paraphrase\n- Do NOT summarize\n- Use full sentences from the text\n- Output ONLY valid JSON\n- No markdown\n- No explanations\n\nTEXT:\nThese rights below are specifically outlined in part 3, Article 38 of the Constitution of Nepal, \n2015, which deals with the rights of women. \nRights of Women According to the Constitution of Nepal, 2015 \n1. Equal Lineage Rights: \no Provision: Every woman has equal rights to lineage without any gender-based \ndiscrimination. \no Explanation: This ensures that women have equal rights to inherit property from \ntheir parents and are not discriminated against based on their gender.\n\nJSON FORMAT:\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of str

In [20]:
# IMPORTANT: final_prompt is a PromptValue
prompt_text = final_prompt.to_string()

In [21]:
response = llm.invoke(prompt_text)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [22]:
print(response)


JSON INSTANCE:
```
{
  "qna_pairs": [
    {
      "question": "What is the provision regarding equal lineage rights for women in the Constitution of Nepal, 2015?",
      "answer": "Every woman has equal rights to lineage without any gender-based discrimination."
    },
    {
      "question": "What does the provision for equal lineage rights ensure for women in the Constitution of Nepal, 2015?",
      "answer": "This ensures that women have equal rights to inherit property from their parents and are not discriminated against based on their gender."
    },
    {
      "question": "What article of the Constitution of Nepal, 2015 deals with the rights of women?",
      "answer": "Rights of Women According to the Constitution of Nepal, 2015"
    },
    {
      "question": "What part of the Constitution of Nepal, 2015 does part 3, Article 38 belong to?",
      "answer": "part 3"
    },
    {
      "question": "What is the gender-based discrimination that women are protected against in the 

In [23]:
import json
import re

def extract_json_from_text(text: str) -> dict:
    """
    Extracts the first valid JSON object from model output.
    Handles markdown blocks and raw JSON.
    """
    # Remove markdown ``` blocks
    text = re.sub(r"```(?:json)?", "", text)
    text = text.replace("```", "").strip()

    # Find first JSON object
    match = re.search(r"\{.*\}", text, re.DOTALL)
    if not match:
        raise ValueError("No JSON object found")

    json_str = match.group(0)

    return json.loads(json_str)


In [24]:
res = extract_json_from_text(response)

In [25]:
print(res)

{'qna_pairs': [{'question': 'What is the provision regarding equal lineage rights for women in the Constitution of Nepal, 2015?', 'answer': 'Every woman has equal rights to lineage without any gender-based discrimination.'}, {'question': 'What does the provision for equal lineage rights ensure for women in the Constitution of Nepal, 2015?', 'answer': 'This ensures that women have equal rights to inherit property from their parents and are not discriminated against based on their gender.'}, {'question': 'What article of the Constitution of Nepal, 2015 deals with the rights of women?', 'answer': 'Rights of Women According to the Constitution of Nepal, 2015'}, {'question': 'What part of the Constitution of Nepal, 2015 does part 3, Article 38 belong to?', 'answer': 'part 3'}, {'question': 'What is the gender-based discrimination that women are protected against in the Constitution of Nepal, 2015?', 'answer': 'discrimination based on their gender in inheriting property from their parents'}]

In [29]:
len(docs)

82

In [30]:
START_INDEX = 0  # resume from chunk 18

In [31]:
import json

with open("QAPairs.jsonl", "a", encoding="utf-8") as f:
    for i in range(START_INDEX, 5):  # change 2 → len(docs) later

        # 1. Build prompt
        final_prompt = prompt.invoke({
            "chunk": docs[i].page_content
        })

        prompt_text = final_prompt.to_string()

        # 2. Call LLM
        raw_output = llm.invoke(prompt_text)

        # 3. Extract JSON safely
        try:
            parsed_json = extract_json_from_text(raw_output)
        except Exception as e:
            print(f"JSON extraction failed at chunk {i}: {e}")
            continue

        # 4. Validate schema manually
        if "qna_pairs" not in parsed_json:
            print(f"Invalid schema at chunk {i}")
            continue

        # 5. Write JSONL
        for qa in parsed_json["qna_pairs"]:
            record = {
                "messages": [
                    {"role": "user", "content": qa["question"].strip()},
                    {"role": "assistant", "content": qa["answer"].strip()}
                ]
            }
            f.write(json.dumps(record, ensure_ascii=False) + "\n")

        print(f"✅ Successfully processed chunk {i}")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


✅ Successfully processed chunk 0


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


✅ Successfully processed chunk 1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


✅ Successfully processed chunk 2


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


✅ Successfully processed chunk 3
✅ Successfully processed chunk 4


In [32]:
!pip freeze -> requirements.txt